<a href="https://colab.research.google.com/github/farieu/data-analysis/blob/Pipeline/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Pipeline de Pré-Processamento

#### Imputação de bibliotecas

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder, MultiLabelBinarizer, FunctionTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
import numpy as np

In [4]:
df = pd.read_csv('/content/drive/MyDrive/BackEnd/GoodReads_100k_books.csv')

Testando uma tentativa de Pipeline de pré-processamento, para codificar e utilizar o RandomForest.

In [5]:
df.dropna(subset=['title', 'desc', 'genre', 'bookformat'], inplace=True)
numeric_features = ['pages', 'totalratings', 'reviews']
categorical_features = ['author', 'bookformat', 'genre', 'title']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

Aplicando o pipeline e separando as variáveis independentes e a variável alvo.

In [6]:
X = df.drop(columns=['rating', 'desc', 'isbn', 'isbn13', 'img', 'link'])
y = df['rating']

df_preprocessed = pipeline.fit_transform(X)
print(X.shape)

df_preprocessed_df = pd.DataFrame(df_preprocessed.toarray() if isinstance(df_preprocessed, np.ndarray) else df_preprocessed)


(84054, 7)


Separando conjunto de dados processado pelo pipeline em 80% treino e 20% teste

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_preprocessed, y, test_size=0.2, random_state=42)

In [8]:
print(f'Tamanho do conjunto de treino: {X_train.shape}')
print(f'Tamanho do conjunto de teste: {X_test.shape}')

Tamanho do conjunto de treino: (67243, 208839)
Tamanho do conjunto de teste: (16811, 208839)


O shape do conjunto ficou dividido do mesmo tamanho que o modelo de RL.